In [ ]:
from __future__ import division
from __future__ import print_function
from operator import itemgetter
from itertools import combinations
import time
import os
import numpy as np
import networkx as nx
import scipy.sparse as sp
import scipy.io as sio
from sklearn import metrics
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import pickle
import torch

In [ ]:
# gene interaction network
gene_phenes_path = '/PGCN/data_prioritization/genes_phenes.mat'
f = h5py.File(gene_phenes_path, 'r')
gene_network_adj = sp.csc_matrix((np.array(f['GeneGene_Hs']['data']),
    np.array(f['GeneGene_Hs']['ir']), np.array(f['GeneGene_Hs']['jc'])),
    shape=(12331,12331))
gene_network_adj = gene_network_adj.tocsr()
gene_network_adj_row = (gene_network_adj.tocoo()).row
gene_network_adj_col = (gene_network_adj.tocoo()).col
gene_network_adj_value = np.array(f['GeneGene_Hs']['data'])

disease_network_adj = sp.csc_matrix((np.array(f['PhenotypeSimilarities']['data']),
    np.array(f['PhenotypeSimilarities']['ir']), np.array(f['PhenotypeSimilarities']['jc'])),
    shape=(3215, 3215))
disease_network_adj = disease_network_adj.tocsr()

disease_network_adj_row = (disease_network_adj.tocoo()).row
disease_network_adj_col = (disease_network_adj.tocoo()).col
disease_network_adj_value = np.array(f['PhenotypeSimilarities']['data'])

# New Section

In [ ]:
# gene disease network
dg_ref = f['GenePhene'][0][0]
gene_disease_adj = sp.csc_matrix((np.array(f[dg_ref]['data']),
    np.array(f[dg_ref]['ir']), np.array(f[dg_ref]['jc'])),
    shape=(12331, 3215))
gene_disease_adj = gene_disease_adj.tocsr()


In [ ]:
# novel disease network
novel_associations_adj = sp.csc_matrix((np.array(f['NovelAssociations']['data']),
    np.array(f['NovelAssociations']['ir']), np.array(f['NovelAssociations']['jc'])),
    shape=(12331,3215))
novel_associations_adj_row = (novel_associations_adj.tocoo()).row
novel_associations_adj_col = (novel_associations_adj.tocoo()).col
novel_associations_adj_values = np.array(f['NovelAssociations']['data'])

In [ ]:
disease_tfidf_path = '/PGCN/data_prioritization/clinicalfeatures_tfidf.mat'
f_disease_tfidf = h5py.File(disease_tfidf_path,"r")
disease_tfidf = np.array(f_disease_tfidf['F'])
disease_tfidf = np.transpose(disease_tfidf)
disease_tfidf = sp.csc_matrix(disease_tfidf)

In [ ]:
gene_feature_path = '/PGCN/data_prioritization/GeneFeatures.mat'
f_gene_feature = h5py.File(gene_feature_path,'r')
gene_feature_exp = np.array(f_gene_feature['GeneFeatures'])
gene_feature_exp = np.transpose(gene_feature_exp)
gene_network_exp = sp.csc_matrix(gene_feature_exp)

In [ ]:
# Gene feature2:other species features
row_list = [3215, 1137, 744, 2503, 1143, 324, 1188, 4662, 1243]
gene_feature_list_other_spe = list()
for i in range(1,9):
    dg_ref = f['GenePhene'][i][0]
    disease_gene_adj_tmp = sp.csc_matrix((np.array(f[dg_ref]['data']),
        np.array(f[dg_ref]['ir']), np.array(f[dg_ref]['jc'])),
        shape=(12331, row_list[i]))
    gene_feature_list_other_spe.append(disease_gene_adj_tmp)

In [ ]:
gene_feat = sp.hstack(gene_feature_list_other_spe+[gene_feature_exp])

dis_feat = disease_tfidf

In [ ]:
### Create the gene disease network from gene network and disease network
# gene Network
gene_network_adj_row = (gene_network_adj.tocoo()).row
gene_network_adj_col = (gene_network_adj.tocoo()).col
gene_network_adj_value = np.array(f['GeneGene_Hs']['data'])
gene_network_matrix = np.zeros((np.shape(gene_network_adj)))
for i,j in zip(gene_network_adj_row,gene_network_adj_col):
    gene_network_matrix[i,j] = 1

In [ ]:
def network_edge_threshold(network_adj, threshold):
    edge_tmp, edge_value, shape_tmp = sparse_to_tuple(network_adj)
    preserved_edge_index = np.where(edge_value>threshold)[0]
    preserved_network = sp.csr_matrix(
        (edge_value[preserved_edge_index],
        (edge_tmp[preserved_edge_index,0], edge_tmp[preserved_edge_index, 1])),
        shape=shape_tmp)
    return preserved_network

In [ ]:
def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

In [ ]:
# disease Network
disease_network_adj = network_edge_threshold(disease_network_adj,0.2)
disease_network_adj_row = (disease_network_adj.tocoo()).row
disease_network_adj_col = (disease_network_adj.tocoo()).col
disease_network_adj_value = np.array(f['PhenotypeSimilarities']['data'])
disease_network_matrix = np.zeros((np.shape(disease_network_adj)))
for i,j in zip(disease_network_adj_row,disease_network_adj_col):
    disease_network_matrix[i,j] = 1
disease_network_matrix = disease_network_matrix - np.diag(np.ones(len(disease_network_matrix)))

In [ ]:
# gene-disease network
novel_associations_adj_row = (novel_associations_adj.tocoo()).row
novel_associations_adj_col = (novel_associations_adj.tocoo()).col
novel_associations_adj_values = np.array(f['NovelAssociations']['data'])
disease_gene_matrix = np.zeros((np.shape(novel_associations_adj)))
for i,j in zip(novel_associations_adj_row,novel_associations_adj_col):
    disease_gene_matrix[i,j] = 1

comb1 = np.hstack((gene_network_matrix,disease_gene_matrix))
comb2 = np.hstack((disease_gene_matrix.T,disease_network_matrix))
comb = np.vstack((comb1,comb2))

In [ ]:
### Combine the fetaure matrix
disease_network_adj_row = (disease_network_adj.tocoo()).row
disease_network_adj_col = (disease_network_adj.tocoo()).col
disease_network_adj_value = np.array(f['PhenotypeSimilarities']['data'])
disease_network_matrix = np.zeros((np.shape(disease_network_adj)))
for i,j in zip(disease_network_adj_row,disease_network_adj_col):
    disease_network_matrix[i,j] = 1
disease_network_matrix = disease_network_matrix - np.diag(np.ones(len(disease_network_matrix)))

In [ ]:
# edge_index, label and the feature matrix that will be used
num_disease = 3215
num_gene = 12331
edge_index = torch.tensor(comb.nonzero())
label = torch.zeros(num_disease+num_gene)
label[0:num_gene] = 1
disease_feat_matrix = dis_feat.tocsc().toarray()
gene_feat_matrix = gene_feat.tocsc().toarray()
disease_feat_none = np.zeros((num_gene,np.shape(disease_feat_matrix)[1]))
gene_feat_none = np.zeros((num_disease,np.shape(gene_feat_matrix)[1]))

<ipython-input-18-4852f0d96649>:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  edge_index = torch.tensor(comb.nonzero())


In [ ]:
import numpy as np
feature_matrix = np.vstack((np.hstack((gene_feat_matrix, disease_feat_none)), np.hstack(
    (gene_feat_none, disease_feat_matrix))))
feature_matrix = torch.from_numpy(feature_matrix).double()

In [ ]:
pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.2 MB/s eta 0:00:00


In [ ]:
from torch_geometric.datasets import Entities,Flickr
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Linear
from torch.nn import BatchNorm1d
from torch.utils.data import Dataset
from torch_geometric.nn import GCNConv
from torch_geometric.nn import ChebConv
from torch_geometric.nn import global_add_pool, global_mean_pool
from torch_geometric.data import DataLoader
# from torch_scatter import scatter_mean

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
import matplotlib as mpl
import matplotlib.pyplot as plt

from torch_geometric.data import InMemoryDataset
from torch_geometric.data import Data
import os.path as osp
from torch_geometric.data import Dataset
from torch_geometric.data import NeighborSampler
from torch_geometric.data import Batch, ClusterData, ClusterLoader, DataLoader

from torch_geometric.utils import (negative_sampling, remove_self_loops,
                                   add_self_loops, structured_negative_sampling,train_test_split_edges)

from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, ChebConv, RGCNConv  # noqa
from torch_geometric.nn import Node2Vec

import math
import random
from torch_geometric.utils import to_undirected
from torch_geometric.data import GraphSAINTRandomWalkSampler, GraphSAINTSampler
from torch_geometric.utils import degree
from torch_geometric.nn import FastRGCNConv

import copy

# from .utils import DiseaseGeneDataset,train_test_split_edges


In [ ]:
class DiseaseGeneDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(DiseaseGeneDataset, self).__init__(root, transform,
                                                 pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return ['data.pt']

    def download(self):
        pass

    def process(self):

        data_list = []

        data = Data(x=feature_matrix, edge_index=edge_index, y=label)
        data_list.append(data)

        data, slices = self.collate(data_list)
        torch.save((data, slices),self.processed_paths[0])

In [ ]:
def train_test_split_edges(data, val_ratio=0.1, test_ratio=0.1):
    """Splits the edges of a :obj:`torch_geometric.data.Data` object
    into positive and negative train/val/test edges, and adds attributes of
    `train_pos_edge_index`, `train_neg_adj_mask`, `val_pos_edge_index`,
    `val_neg_edge_index`, `test_pos_edge_index`, and `test_neg_edge_index`
    to :attr:`data`.
    Args:
        data (Data): The data object.
        val_ratio (float, optional): The ratio of positive validation
            edges. (default: :obj:`0.05`)
        test_ratio (float, optional): The ratio of positive test
            edges. (default: :obj:`0.1`)
    :rtype: :class:`torch_geometric.data.Data`
    """

    assert 'batch' not in data  # No batch-mode.

    num_nodes = data.num_nodes
    row, col = data.edge_index
    data.edge_index = None

    # Return upper triangular portion.
    mask = row < col
    row, col = row[mask], col[mask]

    n_v = int(math.floor(val_ratio * row.size(0)))
    n_t = int(math.floor(test_ratio * row.size(0)))

    # Positive edges.
    perm = torch.randperm(row.size(0))
    row, col = row[perm], col[perm]


    r, c = row[:n_v], col[:n_v]
    data.val_pos_edge_index = torch.stack([r, c], dim=0)
    r, c = row[n_v:n_v + n_t], col[n_v:n_v + n_t]
    data.test_pos_edge_index = torch.stack([r, c], dim=0)

    r, c = row[n_v + n_t:], col[n_v + n_t:]
    data.train_pos_edge_index = torch.stack([r, c], dim=0)
    data.train_pos_edge_index = to_undirected(data.train_pos_edge_index)

    # Negative edges.
    neg_adj_mask = torch.ones(num_nodes, num_nodes, dtype=torch.uint8)
    neg_adj_mask = neg_adj_mask.triu(diagonal=1).to(torch.bool)
    neg_adj_mask[row, col] = 0

    neg_row, neg_col = neg_adj_mask.nonzero(as_tuple=False).t()
    perm = torch.randperm(neg_row.size(0))[:n_v + n_t]
    neg_row, neg_col = neg_row[perm], neg_col[perm]

    neg_adj_mask[neg_row, neg_col] = 0
    data.train_neg_adj_mask = neg_adj_mask

    row, col = neg_row[:n_v], neg_col[:n_v]
    data.val_neg_edge_index = torch.stack([row, col], dim=0)

    row, col = neg_row[n_v:n_v + n_t], neg_col[n_v:n_v + n_t]
    data.test_neg_edge_index = torch.stack([row, col], dim=0)

    return data

In [ ]:
num_nodes = 15546
num_features = 37287

num_gene = 12331
num_Disease = 3215

In [ ]:
# feature_matrix_np = feature_matrix.numpy()
# np.save("feature_matrix.txt.npy", feature_matrix_np)

In [ ]:
# feature_matrix = np.load("./feature_matrix.txt.npy")
# label = np.load("./label.txt.npy")
# edge_index = np.load("./edge_index.txt.npy")

In [ ]:
feature_matrix = feature_matrix.reshape((num_nodes,-1))
edge_index = edge_index.reshape((2,-1))

In [ ]:
feature_matrix = feature_matrix.numpy()

In [ ]:
label = label.numpy()

In [ ]:
edge_index = edge_index.numpy()

In [ ]:
feature_matrix = torch.from_numpy(feature_matrix)
label = torch.from_numpy(label).long()
edge_index = torch.from_numpy(edge_index)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
dataset_gd = DiseaseGeneDataset(root = "/scratch/gilbreth/shu30/GraphDropout/disease_gene.dataset")

Processing...
Done!


In [ ]:
# import torch
# import torch.nn.functional as F
# from torch_geometric.nn import GATConv
# from torch_geometric.data import Data

# class GATNet(torch.nn.Module):
#     def __init__(self, num_features, num_heads=8, dropout_rate=0.6):
#         super(GATNet, self).__init__()
#         self.conv1 = GATConv(num_features, 8, heads=num_heads, dropout=dropout_rate)
#         self.conv2 = GATConv(8 * num_heads, 1, heads=1, dropout=dropout_rate)
#         self.dropout_rate = dropout_rate  # Store as an instance variable

#     def forward(self, x, edge_index):
#         x = F.dropout(x, p=self.dropout_rate, training=self.training)
#         x = F.elu(self.conv1(x, edge_index))
#         x = F.dropout(x, p=self.dropout_rate, training=self.training)
#         x = self.conv2(x, edge_index)
#         return x


# feature_matrix = feature_matrix.float()
# edge_index = edge_index.long()

# # Validate edge_index here
# num_nodes = feature_matrix.size(0)
# if edge_index.max().item() >= num_nodes or edge_index.min().item() < 0:
#     raise ValueError(f"Edge index out of range. Valid index range: [0, {num_nodes - 1}], but got range [{edge_index.min().item()}, {edge_index.max().item()}].")

# num_features = feature_matrix.shape[1]
# model = GATNet(num_features).float()

# label = label.float()  # Ensure label is float
# data = Data(x=feature_matrix, edge_index=edge_index, y=label)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)
# data = data.to(device)

# criterion = torch.nn.BCEWithLogitsLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

# for epoch in range(200):
#     model.train()
#     optimizer.zero_grad()
#     out = model(data.x.to(device), data.edge_index.to(device))
#     loss = criterion(out.squeeze(), data.y.to(device))
#     loss.backward()
#     optimizer.step()

#     if epoch % 10 == 0:
#         print(f'Epoch {epoch+1}, Loss: {loss.item()}')



In [ ]:
data = dataset_gd.data
data.num_classes = 2
data.num_relations = 3
data = train_test_split_edges(data, val_ratio=0.05, test_ratio=0.1)
data_dropout = copy.deepcopy(data)

/usr/local/lib/python3.10/dist-packages/torch_geometric/data/in_memory_dataset.py:284: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [ ]:
edge_index = data.train_pos_edge_index
num_gene = 12331
gg_index = []
dd_index = []
gd_index = []
for i in range(edge_index.shape[1]):
    if (edge_index[:,i][0] < num_gene and edge_index[:,i][1] >= num_gene) or (edge_index[:,i][0] >= num_gene and edge_index[:,i][1] <= num_gene):
        gd_index.append(i)
    elif (edge_index[:,i][0] < num_gene and edge_index[:,i][1] < num_gene):
        gg_index.append(i)
    else:
        dd_index.append(i)

In [ ]:
edge_type_gd = torch.zeros(edge_index.shape[1]).long()
edge_type_gd[gd_index] = torch.zeros(len(gd_index)).long()
edge_type_gd[gg_index] = torch.ones(len(gg_index)).long()
edge_type_gd[dd_index] = 2 * torch.ones(len(dd_index)).long()

In [ ]:
data.edge_type = edge_type_gd
data_dropout.edge_type = edge_type_gd

In [44]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = RGATConv(data.num_features, 32, data.num_relations,
                              num_bases=30)
        self.conv2 = RGATConv(32, 16, data.num_relations,
                              num_bases=30)
        self.fc1 = Linear(16, 16)

        self.Dropconv1 = GCNConv(data.num_features, 16, cached=False).to(torch.double)
        self.Dropconv2 = GCNConv(16, 16, cached=False).to(torch.double)
        self.Dropfc1 = Linear(16, 16).to(torch.double)

    def reparameterize(self,logvar):
        std = torch.exp(-0.5*logvar)
        return std

    def getvar(self):
        x = F.relu(self.Dropconv1(data.x, data.train_pos_edge_index))
        x = self.Dropconv2(x, data.train_pos_edge_index)
        x = self.Dropfc1(x)
        return x

    def GCN(self,z=None):

        y = F.relu(self.conv1(data.x, data.train_pos_edge_index, data.edge_type))
        y = self.conv2(y, data.train_pos_edge_index, data.edge_type)
        y = self.fc1(y)
        if TRAIN:
            y_out = y.mul(torch.randn(y.size()).to(device).mul(z)+1)
        else:
            y_out = y
        return y_out


    def forward(self, pos_edge_index, neg_edge_index):
        if TRAIN:
            z = self.reparameterize(self.getvar())
            out = self.GCN(z)
            total_edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=-1)
            x_j = torch.index_select(out, 0, total_edge_index[0])
            x_i = torch.index_select(out, 0, total_edge_index[1])
            return torch.einsum("ef,ef->e", x_i, x_j), z, out
        else:
            out = self.GCN()
            total_edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=-1)
            x_j = torch.index_select(out, 0, total_edge_index[0])
            x_i = torch.index_select(out, 0, total_edge_index[1])
            return torch.einsum("ef,ef->e", x_i, x_j), None, out  # Return None for the second value

In [ ]:
class My_loss(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x, y, y_d):
        loss = F.binary_cross_entropy_with_logits(x, y)
        if y_d is not None:
            loss += 0.000001 * torch.sum(torch.sum(torch.log(y_d)**2))
        return loss

def get_link_labels(pos_edge_index, neg_edge_index):
    link_labels = torch.zeros(pos_edge_index.size(1) +
                              neg_edge_index.size(1)).float().to(device)
    link_labels[:pos_edge_index.size(1)] = 1.
    return link_labels

def get_dis_gene_edges(edge_indices,edge_type):
    edges = []
    for i in range(edge_indices.shape[1]):
        if (edge_indices[:,i][0] < num_gene and edge_indices[:,i][1] >= num_gene) or (edge_indices[:,i][0] >= num_gene and edge_indices[:,i][1] < num_gene):
            edges.append(i)
    return edge_indices[:,edges],edge_type[edges]

In [ ]:
train_pos_edge_index,train_edge_type = get_dis_gene_edges(data.train_pos_edge_index,data.edge_type)
data.train_pos_edge_index = train_pos_edge_index
data.edge_type = train_edge_type

In [56]:
def train():
    TRAIN = True
    model.train()
    optimizer.zero_grad()

    x, pos_edge_index = data.x, data.train_pos_edge_index

    _edge_index, _ = remove_self_loops(pos_edge_index)
    pos_edge_index_with_self_loops, _ = add_self_loops(_edge_index,
                                                       num_nodes=x.size(0))

    neg_edge_index = negative_sampling(
        edge_index=pos_edge_index_with_self_loops, num_nodes=x.size(0),
        num_neg_samples=3 * pos_edge_index.size(1))

    neg_edge_type = torch.zeros(neg_edge_index.shape[1]).long()
    neg_edge_index, neg_edge_type = get_dis_gene_edges(neg_edge_index,neg_edge_type)

    link_logits,y_d,node_embedding = model(pos_edge_index, neg_edge_index)
    link_labels = get_link_labels(pos_edge_index, neg_edge_index)

    #print("TrainROC:",roc_auc_score(link_labels.detach().cpu(), torch.sigmoid(link_logits.detach().cpu())),"TrainPRC:",average_precision_score(link_labels.detach().cpu(), torch.sigmoid(link_logits.detach().cpu())))

    criterion = My_loss()
    loss = criterion(link_logits, link_labels, y_d)
    loss.backward()
    optimizer.step()

    return loss


In [54]:
from torch_geometric.nn import RGATConv
model, data, data_dropout = Net().to(device), data.to(device), data_dropout.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0005)
model = model.double()
test_pos = []
for i in range(data.test_pos_edge_index.shape[1]):
    if (data.test_pos_edge_index[:,i][0] < num_gene and data.test_pos_edge_index[:,i][1] >= num_gene) or (data.test_pos_edge_index[:,i][0] >= num_gene and data.test_pos_edge_index[:,i][1] <= num_gene):
        test_pos.append(i)
test_pos_edge_index = data.test_pos_edge_index[:,test_pos].to(device)

test_neg = []
for i in range(data.test_neg_edge_index.shape[1]):
    if (data.test_neg_edge_index[:,i][0] < num_gene and data.test_neg_edge_index[:,i][1] >= num_gene) or (data.test_neg_edge_index[:,i][0] >= num_gene and data.test_neg_edge_index[:,i][1] <= num_gene):
        test_neg.append(i)
test_neg_edge_index = data.test_neg_edge_index[:,test_neg].to(device)

@torch.no_grad()
def test():
    TRAIN = False
    model.eval()
    link_probs = torch.sigmoid(model(test_pos_edge_index, test_neg_edge_index[:,range(test_pos_edge_index.shape[1])])[0])
    link_labels = get_link_labels(test_pos_edge_index, test_neg_edge_index[:,range(test_pos_edge_index.shape[1])])
    link_probs = link_probs.detach().cpu().numpy()
    link_labels = link_labels.detach().cpu().numpy()
    #print("TestROC:",roc_auc_score(link_labels, link_probs))
    #print("TestPRC:",average_precision_score(link_labels, link_probs))

    return link_labels, link_probs

best_val_perf = test_perf = 0



In [57]:
for epoch in range(1, 700):
    TRAIN = True
    train_loss = train()
    TRAIN = False
    labels, probs = test()